In [2]:
import pandas as pd
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import np_utils

train_data = pd.read_csv(r'/mnt/ML/train.csv')
test_data = pd.read_csv(r'/mnt/ML/testA.csv')

train_get_feature = [k.split(',') for k in train_data.heartbeat_signals]
test_get_feature = [k.split(',') for k in test_data.heartbeat_signals]

train_feature = pd.DataFrame(train_get_feature)
train_feature = train_feature.astype(float)

test_feature = pd.DataFrame(test_get_feature)
test_feature = test_feature.astype(float)


Using TensorFlow backend.


In [3]:
X_train, X_validation, Y_train, Y_validation = train_test_split(train_feature, train_data.label, test_size=0.2,random_state=42)

train_X = np.array(X_train)
train_Y = np.array(Y_train)
X_validation = np.array(X_validation)
Y_validation = np.array(Y_validation)


train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], 1)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(8,), padding='same',
                           activation='relu', input_shape=(train_X.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(4,), strides=2, padding='same'),

    tf.keras.layers.Conv1D(filters=64, kernel_size=(8,), padding='same',
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(4,), strides=2, padding='same'),
    
    tf.keras.layers.Conv1D(filters=128, kernel_size=(6,), padding='same',
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(3,), strides=2, padding='same'),

    tf.keras.layers.Conv1D(filters=128, kernel_size=(4,), padding='same',
                           activation='relu'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(3,), strides=1, padding='same'),
                           
    tf.keras.layers.Conv1D(filters=256, kernel_size=(2,), padding='same',
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(3,), strides=1, padding='same'),
    
    tf.keras.layers.Conv1D(filters=512, kernel_size=(2,), padding='same',
                           activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool1D(pool_size=(2,), strides=1, padding='same'),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='softmax')
])


model.summary()
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=1e-6, nesterov=False)

# opt=tf.keras.optimizers.Adam(
#     learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
#     name='Adam'
# )
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

num_epochs = 60
cnn=model.fit(train_X,
          train_Y,
          batch_size=64,
          epochs=num_epochs,
          validation_data=(X_validation,Y_validation),
          verbose=2)


def abs_sum(y_tru, y_pre):
    y_tru = pd.get_dummies(data=y_tru)
    y_pre = np.array(y_pre)
    y_tru = np.array(y_tru)
    loss = sum(sum(abs(y_pre - y_tru)))
    return loss


pre = model.predict(X_validation)
print(abs_sum(Y_validation, pre))



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 205, 64)           576       
_________________________________________________________________
batch_normalization (BatchNo (None, 205, 64)           256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 103, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 103, 64)           32832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 103, 64)           256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 52, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 52, 128)           4

In [5]:
#60baseline 
#60第一次299第二次350
#60复杂网络第一次328第二次285第三次291
#100复杂网络第一次252第二次245第三次（random_state=42）295第四次259(256)第五次280
##100更复杂结构第一次275(round后267)第二次262（round后253）第三次268（256）
#baseline 60 第一次309（306)
print(abs_sum(Y_validation, pre))

202.57862663269043


In [6]:
pre2=pre.copy()
for i in range(len(pre2)):
    for j in range(len(pre2[0])):
        pre2[i][j]=int(pre2[i][j]+0.5)
        
print(abs_sum(Y_validation, pre2))

185.0


In [7]:
test_feature = pd.DataFrame(test_get_feature)
test_feature = test_feature.astype(float)

In [8]:
test_X = np.array(test_feature)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)

In [14]:
pre = model.predict(test_X)
data_test_price = pd.DataFrame(pre,columns = ['label_0','label_1','label_2','label_3'])
results = pd.concat([test_data['id'],data_test_price],axis = 1)
submit_file_z_score = r'/mnt/ML/cnn_A1_202.csv'
results.to_csv(submit_file_z_score,encoding='utf8',index=0)

test_data1 = pd.read_csv(r'/mnt/ML/testB.csv')

test_data1.heartbeat_signals

test_get_feature1 = [k.split(',') for k in test_data1.heartbeat_signals]



test_feature1 = pd.DataFrame(test_get_feature1)
test_feature1 = test_feature1.astype(float)
test_X1 = np.array(test_feature1)
test_X1 = test_X1.reshape(test_X1.shape[0], test_X1.shape[1], 1)
pre1 = model.predict(test_X1)

data_test_price1 = pd.DataFrame(pre1,columns = ['label_0','label_1','label_2','label_3'])
results1 = pd.concat([test_data1['id'],data_test_price1],axis = 1)
submit_file_z_score1 = r'/mnt/ML/cnn_B1_202.csv'
results1.to_csv(submit_file_z_score1,encoding='utf8',index=0)

In [9]:

model.save('cnn_conv8_202', save_format='tf')
 
# 加载模型，通过指定存放模型的文件夹来加载
# new_model = keras.models.load_model('cnn_conv8_194')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cnn_conv8_202/assets
